In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
from vjp.openai_prompt import Prompt
import vjp.data as data

In [ ]:
upheld, rejected = data.load_second_instance_labeled()
df = data.shot_based_dataframe(upheld, rejected)
data_dict = df.to_dict("records")

In [ ]:
prompt = Prompt(data_dict)
messages = prompt.create_prompt()

In [ ]:
for message, label in zip(messages, df['label'].tolist()):
    prompt.print_nice_prompt(messages[0])
    if label == 1:
        print("True result: uphold")
    else:
        print("True result: reject")
    break